<a href="https://colab.research.google.com/github/aekanun2020/2022-airflow/blob/main/etl_dag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from datetime import datetime
from airflow.models.dag import DAG
from airflow.decorators import task
from airflow.utils.task_group import TaskGroup
from airflow.providers.microsoft.mssql.hooks.mssql import MsSqlHook
from airflow.hooks.base_hook import BaseHook
import pandas as pd
from sqlalchemy import create_engine

#transformation tasks
def transform_src():
  raw_pd = pd.read_csv('LoanStats_web.csv')
  selected_pd = raw_pd[['int_rate','loan_status','term']]
  regex_list = [r'%','months']
  selected_pd['int_rate'].replace(regex=regex_list,value='',inplace=True)
  selected_pd['term'].replace(regex=regex_list,value='',inplace=True)
  selected_pd.fillna('N/A',inplace=True)
  selected_pd.to_csv('test.json')
  print('Transform completely')

#DAG
with DAG(dag_id="etl_dag",schedule_interval="0 9 * * *", start_date=datetime(2022, 5, 9),catchup=False,  tags=["etl_test"]) as dag:

    # [START howto_task_group_section_2]
    with TaskGroup("transform_src_group", tooltip="Transform and stage data") as transform_src_group:
        transform_src = transform_src()
        #define task order
        [transform_src]

    transform_src_group